In [ ]:
import time 
from tqdm import tqdm
from nba_api.stats.endpoints import leaguegamelog, boxscoreadvancedv3, boxscorefourfactorsv3
from nba_api.stats.endpoints import boxscorescoringv3,boxscoreplayertrackv3,boxscoremiscv3,boxscorehustlev2
import numpy as np
import pandas as pd
from IPython.display import clear_output
data_DIR = "../Leaderboards/boxscores/"

In [ ]:
def get_gameids(season,name):
    stats = leaguegamelog.LeagueGameLog(player_or_team_abbreviation="T",season=season,season_type_all_star="Regular Season")
    df = stats.get_data_frames()[0]
    game_ids1 = df["GAME_ID"].tolist()
    game_ids1  = np.unique(game_ids1)
    try:
        dfr = pd.read_csv(data_DIR+"NBA_BoxScores_"+name+"_"+season+".csv")
        game_ids3 = dfr["gameId"].astype(str).tolist()
        game_ids2 = ["00" +s for s in game_ids3]
        game_ids2  = np.unique(game_ids2)
        game_ids = list(set(game_ids1).difference(game_ids2))
    except:
        game_ids = game_ids1
    return game_ids

In [ ]:
def get_game_box(game_ids,fun):
    df_ap = []
    for game_id in tqdm(game_ids):
        for ii in range(5):
            time.sleep(0.6)
            try:
                stats = fun(game_id=game_id)
                df1 = stats.get_data_frames()[1]
                df_ap.append(df1)
                break
            except:
                print(game_id)
                continue
    clear_output(wait=True)
    return df_ap

In [ ]:
def get_boxscores(seasons,fun,name):
    for season in seasons:  
        game_ids = get_gameids(season,name)
        try:
            df_ap = get_game_box(game_ids,fun)
            df = pd.concat(df_ap)
            df.to_csv(data_DIR+"NBA_BoxScores_"+name+"_"+season+".csv")
        except:
            continue

In [ ]:
season_start = 2023
season_end = 2023
seasons = np.arange(season_start,season_end+1,1).astype(str)
seasons

In [ ]:
fun = boxscoreadvancedv3.BoxScoreAdvancedV3
name = "Adv"
get_boxscores(seasons,fun,name)

In [ ]:
fun = boxscorefourfactorsv3.BoxScoreFourFactorsV3
name = "4Factor"
get_boxscores(seasons,fun,name)

In [ ]:
fun = boxscorehustlev2.BoxScoreHustleV2
name = "Hustle"
get_boxscores(seasons,fun,name)

In [ ]:
fun = boxscoremiscv3.BoxScoreMiscV3
name = "Misc"
get_boxscores(seasons,fun,name)

In [ ]:
fun = boxscoreplayertrackv3.BoxScorePlayerTrackV3
name = "Track"
get_boxscores(seasons,fun,name)

In [ ]:
fun = boxscorescoringv3.BoxScoreScoringV3
name = "Scoring"
get_boxscores(seasons,fun,name)